In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load and preprocess the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(train_images)

# Define the CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Define the NeuralNetwork class
class NeuralNetwork:
    def __init__(self, input_shape, num_classes):
        self.model = self.build_model(input_shape, num_classes)

    def build_model(self, input_shape, num_classes):
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Flatten())
        model.add(layers.Dropout(0.5))  # Dropout regularization
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def train(self, train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1):
        history = self.model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size), epochs=epochs, validation_data=(test_images, test_labels))
        return history

    def evaluate(self, test_images, test_labels):
        return self.model.evaluate(test_images, test_labels)

    def predict(self, images):
        return self.model.predict(images)

# Create an instance of the NeuralNetwork class
input_shape = (32, 32, 3)  # CIFAR-10 images are 32x32 RGB images
num_classes = 10
nn = NeuralNetwork(input_shape, num_classes)

# Train the neural network
history = nn.train(train_images, train_labels, epochs=50)

# Evaluate the model on the test set
test_loss, test_acc = nn.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Make predictions on a few test images
predictions = nn.predict(test_images[:5])

# Plot the first few test images and their predicted labels
plt.figure(figsize=(10, 4))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(test_images[i])
    plt.title(f'Predicted: {class_names[np.argmax(predictions[i])]}')
    plt.axis('off')
plt.show()

# Provide the image file path
image_path = '/content/bird.jpg'

# Preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((32, 32))  # Resize to match the input shape of the model
    image = np.array(image)  # Convert to numpy array
    image = image.astype('float32') / 255  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

input_image = preprocess_image(image_path)

# Use the model to predict the class of the image
predicted_class = class_names[np.argmax(nn.predict(input_image)[0])]

# Display the image and the predicted class
image = Image.open(image_path)
plt.imshow(image)
plt.title(f'Predicted Class: {predicted_class}')
plt.axis('off')
plt.show()


170498071/170498071 [==============================] - 4s 0us/step
Epoch 1/50
782/782 [==============================] - 100s 124ms/step - loss: 1.7319 - accuracy: 0.3582 - val_loss: 1.4664 - val_accuracy: 0.4534
Epoch 2/50
674/782 [========================>.....] - ETA: 11s - loss: 1.4570 - accuracy: 0.4685